In [29]:
import pandas as pd

parquet_file = 'dane.parquet'
df = pd.read_parquet(parquet_file)
df = df[:10000]
df

,question,answer
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha..."
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha..."
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha..."
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha..."
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha..."
...,...,...
9995,What is the name of Thammasat University's eng...,Institutes of technology with different origin...
9996,What is Thailand's only government-established...,Institutes of technology with different origin...
9997,What year was Suranaree University of Technolo...,Institutes of technology with different origin...
9998,What is the name of the best-known private ins...,Institutes of technology with different origin...


## 3. Analiza działania modelu retriever (bi-enkoder) na wybranym zbiorze danych (np. sentence-transformers/squad dostępny w ramach platformy HuggingFace [2.5 pkt]

https://huggingface.co/datasets/sentence-transformers/squad)

sugeruje się wykorzystanie modelu multi-qa-mpnet-base-dot-v1

In [30]:
from datasets import Dataset

data = Dataset.from_pandas(df)

data = data.map(
    lambda x: {"answer_length": len(x["answer"].split()), "question_length": len(x["question"].split())}
)

data = data.filter(lambda x: x['answer_length'] > 15)

Filter: 100%|██████████| 10000/10000 [00:00<00:00, 410068.54 examples/s]


In [31]:
def concatenate_text(examples):
    return {
        "text": examples["question"]
        + " \n "
        + examples["answer"]
    }


data = data.map(concatenate_text)
data

Map: 100%|██████████| 10000/10000 [00:00<00:00, 29465.93 examples/s]


Dataset({
    features: ['question', 'answer', 'answer_length', 'question_length', 'text'],
    num_rows: 10000
})

In [32]:
data[1000:]

{'question': ['How much did Beyonce initially contribute to the foundation?',
  'How has this foundation changed in recent years?',
  'What foundation did Beyoncé start after Hurricane Katrina?',
  'How much money did Beyoncé contribute at the beginning of her Hurricane Katrina foundation?',
  "What other hurricane did Beyoncé's foundation help with?",
  'Who did Beyonce participate with in the Hope for Haiti Now: A Global Benefit?',
  'Beyonce opened a cosmetology center in what location?',
  "After Osama Bin Laden's death, what single did Beyonce cover?",
  "How much did the T-shirt with Beyonce's image on it make?",
  'What enterprise did Beyonce and her mother start on March 5, 2010?',
  'What charity benefited from the release of the song, God Bless the USA?',
  'What did she participate in with George Clooney?',
  'Which two stars did Beyoncé help with their Haiti Earthquake organization?',
  'What did Beyoncé open at the Brooklyn Phoenix House in 2010?',
  'What Lee Greenwood so

In [33]:
data["question"][5000:]

['What is one purpose of a greenhouse?',
 'What was one of the first uses of a greenhouse?',
 'Where were the first modern greenhouses built?',
 'What is the name of the solar powered car race held every two years?',
 "What was the winner of the World Solar Challenge's average speed in 2007 in km/h?",
 'What is The World Solar Challenge?',
 'When was The World Solar Challenge started?',
 'What was the average speed of a winning solar powered car in 1987?',
 'What was the average speed of a winning solar powered car by 2007?',
 'What are some other similar car races that use solar powered vehicles?',
 'The first practical solar boat was constructed in what year?',
 'When was the first solar powered boat made?',
 'Who first crossed the Pacific ocean using a solar powered boat?',
 'What was the name of the first solar powered boat that crossed the Atlantic ocean?',
 'What altitude did the Solar Riser reach in feet?',
 'What is the name of the aircraft circling the globe in 2015 via solar 

### b
wykonać osadzenia zapytań i dokumentów (kolumny question i
answer) oraz przeprowadzić analizę przykładowych wyszukiwań, tym
razem posługując się innym tutorialem:
https://sbert.net/examples/applications/semantic-search/README.html

In [57]:
import torch
from sentence_transformers import SentenceTransformer


embedder = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

# byly duplikaty
corpus = list(set(data['text']))

# Use "convert_to_tensor=True" to keep the tensors on GPU (if available)
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=False)

# Query sentences:
queries = [
    "Where is the Notre Dame?",
    "Where did the Virgin Mary allegedly appear in 1858?",
    "How old is the oldest structure at Notre Dame?",
    "What is Beyonce's daughter name?",
    "When was Chopin's funeral?",
    "What was the cause of Chopin's death?",
    "What body of water separates the Hudson and East Rivers?",
    "Between New York City and which nearby urban center is the Hudson River classified as an estuary?",
    "Who was the founder of the National Review magazine?", 
]

In [58]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=False)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = embedder.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)

    print("\nQuery:", query)
    print("Top 5 most similar sentences in corpus:")

    for score, idx in zip(scores, indices):
        print(corpus[idx])
        print(f"(Score: {score:.4f})")



Query: Where is the Notre Dame?
Top 5 most similar sentences in corpus:
The school known as Notre Dame is known by a more lengthy name, what is it? 
 The University of Notre Dame du Lac (or simply Notre Dame /ˌnoʊtərˈdeɪm/ NOH-tər-DAYM) is a Catholic research university located adjacent to South Bend, Indiana, in the United States. In French, Notre Dame du Lac means "Our Lady of the Lake" and refers to the university's patron saint, the Virgin Mary. The main campus covers 1,250 acres in a suburban setting and it contains a number of recognizable landmarks, such as the Golden Dome, the "Word of Life" mural (commonly known as Touchdown Jesus), and the Basilica.
(Score: 25.4066)
What type of institution is the Notre Dame? 
 The University of Notre Dame du Lac (or simply Notre Dame /ˌnoʊtərˈdeɪm/ NOH-tər-DAYM) is a Catholic research university located adjacent to South Bend, Indiana, in the United States. In French, Notre Dame du Lac means "Our Lady of the Lake" and refers to the universi

#### Analiza wyników

Wykorzystalismy model SentenceTransformer do wygenerowania osadzeń tekstowych i bazowalismy na podobieństwie cosinusowym.
Mozna stwierdzic, ze w wiekszosci odpowiedzi pasują kontekstowo do zapytań, co wskazuje na poprawne osadzenia semantyczne.


##### Przykłady:
**Query: Where did the Virgin Mary allegedly appear in 1858?** \
3/5 dobrych odpowiedzi zawierających miejsce, o które zapytaliśmy. Reszta podaje informacje, które równiez dotyczą objawienia Virgin Mary, ale nie tego, o które pytaliśmy. 

**Query: What is Beyonce's daughter name?** \
Tylko jedno z 5 najblizszych odpowiedzi zawiera imię córki Beyonce, reszta to informacje o niej samej.

**Query: Where is the Notre Dame?** \
Wszystkie odpowiedzi pasują.

**Query: Between New York City and which nearby urban center is the Hudson River classified as an estuary?** \
4/5 zawiera odpowiedź na pytanie.

### c. 
należy dodać indeks wyszukiwania FAISS w celu dopasowania
dokumentów do zapytań za pomocą osadzeń i ponownie wykonać
wyszukiwanie (ustalić top_k = 5)

In [61]:
import faiss
import numpy as np

# Create a FAISS index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)

top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=False)
    query_embedding_np = np.array(query_embedding).reshape(1, -1)

    # Search the FAISS index
    distances, indices = index.search(query_embedding_np, top_k)

    print("\nQuery:", query)
    print("Top 5 most similar sentences in corpus:")

    for distance, idx in zip(distances[0], indices[0]):
        print(corpus[idx]) 
        print(f"(Distance: {distance:.4f})")


Query: Where is the Notre Dame?
Top 5 most similar sentences in corpus:
How large is Notre Dame in acres? 
 The University of Notre Dame du Lac (or simply Notre Dame /ˌnoʊtərˈdeɪm/ NOH-tər-DAYM) is a Catholic research university located adjacent to South Bend, Indiana, in the United States. In French, Notre Dame du Lac means "Our Lady of the Lake" and refers to the university's patron saint, the Virgin Mary. The main campus covers 1,250 acres in a suburban setting and it contains a number of recognizable landmarks, such as the Golden Dome, the "Word of Life" mural (commonly known as Touchdown Jesus), and the Basilica.
(Distance: 24.3512)
What type of institution is the Notre Dame? 
 The University of Notre Dame du Lac (or simply Notre Dame /ˌnoʊtərˈdeɪm/ NOH-tər-DAYM) is a Catholic research university located adjacent to South Bend, Indiana, in the United States. In French, Notre Dame du Lac means "Our Lady of the Lake" and refers to the university's patron saint, the Virgin Mary. The

### d
zaimplementować wybraną metrykę oceny skuteczności wyszukiwania

In [67]:
#pzrykład ze strony
# relevant results for query #1, #2, and #3
# actual_relevant = [
#     [2, 4, 5, 7],
#     [1, 4, 5, 7],
#     [5, 8]
# ]


def calculate_mrr(actual_relevant):

    # number of queries
    Q = len(actual_relevant)

    # calculate the reciprocal of the first actual relevant rank
    cumulative_reciprocal = 0
    for i in range(Q):
        if len(actual_relevant[i]) != 0:
            first_result = actual_relevant[i][0]
            reciprocal = 1 / first_result
            cumulative_reciprocal += reciprocal
            print(f"query #{i+1} = 1/{first_result} = {reciprocal}")
        else:
            print(f"query #{i+1} = 0")

    # calculate mrr
    mrr = 1/Q * cumulative_reciprocal

    # generate results
    print("MRR =", round(mrr,2))


### e

porównać wyniki uzyskane bez wykorzystania indeksu i wraz z
wykorzystaniem indeksu FAISS.

In [59]:
len(queries)

9

#### without faiss

In [63]:
actual_relevant = [
    [1,2,3,4,5],
    [1, 2, 4],
    [1],
    [2],
    [1,2,3],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,3,4,5],
    [1,2,3,4]
]
calculate_mrr(actual_relevant)

query #1 = 1/1 = 1.0
query #2 = 1/1 = 1.0
query #3 = 1/1 = 1.0
query #4 = 1/2 = 0.5
query #5 = 1/1 = 1.0
query #6 = 1/1 = 1.0
query #7 = 1/1 = 1.0
query #8 = 1/1 = 1.0
query #9 = 1/1 = 1.0
MRR = 0.94


#### with faiss

In [69]:
actual_relevant_faiss = [
    [1,2,3,4,5],
    [1, 3, 5],
    [1],
    [],
    [1, 4, 5],
    [1, 3, 5],
    [1,2,3,5],
    [1, 5],
    [1,2,3,4]
]

calculate_mrr(actual_relevant_faiss)

query #1 = 1/1 = 1.0
query #2 = 1/1 = 1.0
query #3 = 1/1 = 1.0
query #4 = 0
query #5 = 1/1 = 1.0
query #6 = 1/1 = 1.0
query #7 = 1/1 = 1.0
query #8 = 1/1 = 1.0
query #9 = 1/1 = 1.0
MRR = 0.89


## 4. Analiza działania modelu r e-ranker (cross-enkoder) [1.5 pkt]

### a
do potoku wyszukiwania zaimplementowanego w podpunkcie 2 należy
dodać moduł re-rankingu

In [ ]:
from datasets import load_dataset
from datasets import Dataset

issues_dataset = load_dataset("lewtun/github-issues", split="train")
issues_dataset = issues_dataset.filter(
    lambda x: (x["is_pull_request"] == False and len(x["comments"]) > 0)
)
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset.set_format("pandas")
df = issues_dataset[:]
comments_df = df.explode("comments", ignore_index=True)
comments_dataset = Dataset.from_pandas(comments_df)
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split())}
)
comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)


In [77]:
from transformers import AutoTokenizer, AutoModel
import torch

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

device = torch.device("cpu")
model.to(device)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)


embedding = get_embeddings(comments_dataset["text"][0])
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)
embeddings_dataset.add_faiss_index(column="embeddings")



questions = [
    "What is Elastic Search?",
    "What to do when 'NoneType' object is not callable?",
    "What does get_indices_from_mask_function() return?",
    "What is the max length of file name?",
    "How to import datasets?",
    "How to make pancakes?"
]

In [76]:
from sentence_transformers import CrossEncoder

# Załaduj model cross-encoder
cross_encoder = CrossEncoder("cross-encoder/ms-marco-TinyBERT-L-2-v2")


import pandas as pd

# Funkcja do wykonania re-rankingu
def rerank_with_cross_encoder(question, samples_df):
    # Stworzenie pary (zapytanie, dokument) do oceny przez cross-encoder
    texts = [(question, row["comments"]) for _, row in samples_df.iterrows()]
    
    # Uzyskanie ocen z cross-encodera
    scores = cross_encoder.predict(texts)
    
    # Dodanie wyników do DataFrame
    samples_df["rerank_scores"] = scores
    
    # Posortowanie wyników na podstawie ocen cross-encoder
    samples_df = samples_df.sort_values("rerank_scores", ascending=False)
    
    return samples_df


for question in questions:
    print("QUESTION", question)
    
    # Wyszukiwanie najbliższych sąsiadów z FAISS
    question_embedding = get_embeddings([question]).cpu().detach().numpy()
    scores, samples = embeddings_dataset.get_nearest_examples("embeddings", question_embedding, k=5)
    
    # Tworzenie DataFrame z wynikami wyszukiwania
    samples_df = pd.DataFrame.from_dict(samples)
    samples_df["scores"] = scores
    
    # Sortowanie wyników na podstawie podobieństwa (FAISS)
    samples_df.sort_values("scores", ascending=False, inplace=True)
    
    # Re-ranking wyników przy pomocy cross-encodera
    reranked_samples_df = rerank_with_cross_encoder(question, samples_df)
    
    # Wyświetlenie najlepszych wyników po re-ranking
    for _, row in reranked_samples_df.iterrows():
        print(f"COMMENT: {row.comments}")
        print(f"RE-RANKED SCORE: {row.rerank_scores}")
        print(f"TITLE: {row.title}")
        print(f"URL: {row.html_url}")
        print()

    print("=" * 50)



QUESTION What is Elastic Search?
COMMENT: Thanks for the clarification.

I am not familiar with ElasticSearch, but if I understand well you're trying to migrate your data along with the ES index.
My advice would be to check out ES documentation, for instance, this might help you: https://www.elastic.co/guide/en/cloud/current/ec-migrate-data.html

Let me know if it helps
RE-RANKED SCORE: -5.655293941497803
TITLE: ❓ Sharing ElasticSearch indexed dataset 
URL: https://github.com/huggingface/datasets/issues/1324

COMMENT: Hi !
I tried your code on my side and I was able to workaround this issue by waiting a few seconds before querying the index.
Maybe this is because the index is not updated yet on the ElasticSearch side ?
RE-RANKED SCORE: -8.645547866821289
TITLE: dataset.search() (elastic) cannot reliably retrieve search results
URL: https://github.com/huggingface/datasets/issues/1758

COMMENT: Hi, is this bug deterministic in your poetry env ? I mean, does it always stop at 90% or is it

### b. 
ponownie wykonać wyszukiwania (top_k = 5) dla wybranych zapytań ze
zbioru i dokonać r e-r ankingu za pomocą cross-enkodera (proszę
wykorzystać jeden z pre-trenowanych modeli:
https://sbert.net/docs/pretrained-models/ce-msmarco.html)

In [79]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=False)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = embedder.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)

    reranked_indices = rerank_with_cross_encoder(query, indices)

    print("\nQuery:", query)
    print("Top 5 most similar sentences in corpus:")

    for score, idx in zip(scores, reranked_indices):
        print(corpus[idx])
        print(f"(Score: {score:.4f})")


AttributeError: 'Tensor' object has no attribute 'iterrows'

### c. 
ponownie ocenić skuteczność wyszukiwania za pomocą ustalonych
metryk oceny i porównać z poprzednio uzyskanymi wynikami.